In [ ]:
from palmerpenguins import penguins
from pandas import get_dummies
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Get Data fixed 3/7


In [ ]:
df = penguins.load_penguins().dropna()

df.head(3)

## Define Model and Fit


In [ ]:
X = get_dummies(df[['bill_length_mm', 'species', 'sex']], drop_first = True)
y = df['body_mass_g']

#context for building the api
print(X)
model = LinearRegression().fit(X, y)

## Get some information


In [ ]:
print(f"R^2 {model.score(X,y)}")
print(f"Intercept {model.intercept_}")
print(f"Columns {X.columns}")
print(f"Coefficients {model.coef_}")

## Attempt at Model Validation


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # 80% training and 20% testing

model = LinearRegression().fit(X_train, y_train)
#Asked Chat GPT on what statistics to run on 
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R-squared (R²): {r2}')

## Chapter 2


In [ ]:
from joblib import dump
dump(model, 'penguin_model.pkl')

In [ ]:
from vetiver import VetiverModel

v = VetiverModel(model=model, model_name='penguin_model', prototype_data=X)

In [ ]:
from pins import board_folder
from vetiver import vetiver_pin_write

# Initialize board folder
model_board = board_folder("finalproj/pins-py", allow_pickle_read=True)

# Write the Vetiver model to the board
vetiver_pin_write(model_board, v)


In [ ]:
from vetiver import VetiverAPI
app = VetiverAPI(v, check_prototype=True)

In [ ]:
#vetiver.prepare_docker(model_board, "penguin_model")

In [ ]:
from vetiver.server import predict, vetiver_endpoint
endpoint = vetiver_endpoint("http://127.0.0.1:8080/predict")
endpoint

In [ ]:
import pandas as pd

# Your existing dictionary
new_penguin_dict = {
    "bill_length_mm": [120],  # Enclosed the values in lists
    "sex_male": [1],          # Assuming '1' for male based on your previous messages
    "species_Gentoo": [1],
    "species_Chinstrap": [0]
}

# Convert the dictionary to a DataFrame
new_penguin_df = pd.DataFrame(new_penguin_dict)

# Now, convert the first row of the DataFrame to a dictionary
record_dict = new_penguin_df.iloc[0].to_dict()


import requests  # Ensure this is installed: pip install requests

# Assuming 'endpoint' is your API endpoint URL
response = requests.post(endpoint, json=record_dict)

# Check the response
if response.ok:
    print("Success:", response.json())
else:
    print("Error:", response.status_code, response.text)

In [ ]:
import duckdb
from palmerpenguins import penguins

con = duckdb.connect('my-db.duckdb')
df = con.execute("SELECT * FROM penguins").fetchdf().dropna()
con.close()

Documentation Statement: I used the following resource : <https://www.geeksforgeeks.org/how-to-do-train-test-split-using-sklearn-in-python/>